# Get Around pricing optimization model

The goal is to build a machine learning model which predicts a daily rental price for car owners, using a dataset gathering past transactions.

We will train different regression models and store their train and test scores for comparison:
- baseline model : multivariate linear regression
- random forest
- XG Boost

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import make_scorer, root_mean_squared_error, mean_squared_error, r2_score, mean_absolute_error

import plotly.express as px
import plotly.graph_objects as go

In [2]:
import warnings

warnings.filterwarnings(
    "ignore",
    message="Found unknown categories in columns"
)

# 1. Explanatory data analysis

## 1.1. Overview and basic statistics

In [3]:
# Import dataset
filename = "../data/get_around_pricing_project.csv"
df = pd.read_csv(filename, sep=",")

In [4]:
print("Display of the first lines of the dataset: ")
display(df.head())

Display of the first lines of the dataset: 


,Unnamed: 0,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101
3,3,Citroën,128035,135,diesel,red,convertible,True,True,False,False,True,True,True,158
4,4,Citroën,97097,160,diesel,silver,convertible,True,True,False,False,False,True,True,183


In [5]:
# Delete the Unnamed:0 column
df = df.drop(columns=['Unnamed: 0'])

In [6]:
print(f"Shape of the dataframe (number of rows, number of columns) : {format(df.shape)}\n")

print("Display of the types of the columns :")
print(df.info())

Shape of the dataframe (number of rows, number of columns) : (4843, 14)

Display of the types of the columns :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4843 entries, 0 to 4842
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   model_key                  4843 non-null   object
 1   mileage                    4843 non-null   int64 
 2   engine_power               4843 non-null   int64 
 3   fuel                       4843 non-null   object
 4   paint_color                4843 non-null   object
 5   car_type                   4843 non-null   object
 6   private_parking_available  4843 non-null   bool  
 7   has_gps                    4843 non-null   bool  
 8   has_air_conditioning       4843 non-null   bool  
 9   automatic_car              4843 non-null   bool  
 10  has_getaround_connect      4843 non-null   bool  
 11  has_speed_regulator        4843 non-null   bool  
 12  winter_

In [7]:
print("Basic statistics: ")
display(df.describe(include = "all").round(1))

Basic statistics: 


,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
count,4843,4843.0,4843.0,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843,4843.0
unique,28,NaN,NaN,4,10,8,2,2,2,2,2,2,2,NaN
top,Citroën,NaN,NaN,diesel,black,estate,True,True,False,False,False,False,True,NaN
freq,969,NaN,NaN,4641,1633,1606,2662,3839,3865,3881,2613,3674,4514,NaN
mean,NaN,140962.8,129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121.2
std,NaN,60196.7,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.6
min,NaN,-64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
25%,NaN,102913.5,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.0
50%,NaN,141080.0,120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.0
75%,NaN,175195.5,135.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0


In [8]:
# Missing values
print("Number of missing values: ")
display(df.isnull().sum())

Number of missing values: 


model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64

In [9]:
# Duplicates
col_without_price = [x for x in df.columns if x != 'rental_price_per_day']

nb_dup = df.duplicated().sum()
nb_dup2 = df.duplicated(col_without_price).sum()

print(f"The dataframe contains {nb_dup} duplicates of all columns, and {nb_dup2} duplicates of cars' characteristics).")

The dataframe contains 0 duplicates of all columns, and 0 duplicates of cars' characteristics).


## 1.2. Univariate analysis

We look at the distribution of each variable in the dataset.  
For numeric variables, we display percentiles, plot histograms and box-plots.
For qualitative variables, we inspect values and plot pies.

### Target variable: daily rental price

In [10]:
display(df['rental_price_per_day'].describe().round(1))

count    4843.0
mean      121.2
std        33.6
min        10.0
25%       104.0
50%       119.0
75%       136.0
max       422.0
Name: rental_price_per_day, dtype: float64

In [11]:
fig = px.box(df,
            x='rental_price_per_day',  
            title=f"Box-plot of daily rental price",
            width=900,
            height=300
            )

fig.update_layout(title_x=0.5)
fig.show()

In [12]:
fig = px.histogram(
        df,
        x='rental_price_per_day',
        title=f"Distribution of daily rental price",
        marginal="box", 
)

mean_val = df['rental_price_per_day'].mean()
fig.add_vline(
    x=mean_val,
    line_dash="dash",
    line_color="red"
)

fig.add_annotation(
    x=mean_val,
    y=1.03,
    yref="y domain",
    showarrow=False,
    xanchor="left",
    text=f"Mean: {mean_val:.0f}",
    font=dict(color="red")
)

fig.update_layout(title_x=0.5, yaxis_title="Frequency")
fig.show()

Half of the cars are rented at a daily price between 104€ and 136€, with an average price around 120€. 
The distribution of daily rental prices has a right long tail which probably corresponds to premium cars (more than 185€).
Conversely, there is a small left tail with very small prices (less than 55€).

### Numerical variables

In [13]:
numeric_cols = ['mileage', 'engine_power']

display(df[numeric_cols].describe().round(1))

,mileage,engine_power
count,4843.0,4843.0
mean,140962.8,129.0
std,60196.7,39.0
min,-64.0,0.0
25%,102913.5,100.0
50%,141080.0,120.0
75%,175195.5,135.0
max,1000376.0,423.0


In [14]:
for var in numeric_cols:
    fig = px.box(df,
                x=var,  
                title=f"Box-plot of {var}",
                width=900,
                height=300
                )

    fig.update_layout(title_x=0.5)
    fig.show()

In [15]:
# Histograms

for var in numeric_cols:
    fig = px.histogram(
            df,
            x=var,
            #nbins=30,
            title=f"Distribution of {var}",
            marginal="box", 
    )

    mean_val = df[var].mean()
    fig.add_vline(
        x=mean_val,
        line_dash="dash",
        line_color="red"
    )

    fig.add_annotation(
        x=mean_val,
        y=1.03,
        yref="y domain",
        showarrow=False,
        xanchor="left",
        text=f"Mean: {mean_val:.0f}",
        font=dict(color="red")
    )

    fig.update_layout(title_x=0.5, yaxis_title="Frequency")
    fig.show()

In [16]:
# Percentiles

for var in numeric_cols:
    print(f"Percentiles of {var}:")
    percentiles = df[var].quantile([0.01, 0.05, 0.25, 0.5, 0.75, 0.90, 0.95, 0.99])
    print(f"{percentiles}\n")

Percentiles of mileage:
0.01     17759.24
0.05     46973.10
0.25    102913.50
0.50    141080.00
0.75    175195.50
0.90    205075.60
0.95    233617.10
0.99    320967.02
Name: mileage, dtype: float64

Percentiles of engine_power:
0.01     85.0
0.05     85.0
0.25    100.0
0.50    120.0
0.75    135.0
0.90    190.0
0.95    210.0
0.99    240.0
Name: engine_power, dtype: float64



In [17]:
# Box-plot of engine_power by car_type
fig = px.box(df,
            x="engine_power",
            y="car_type",
            orientation="h",
            points="outliers",
            color="car_type",
            hover_data=["model_key", "fuel", "mileage"],
            title="Distribution of engine_power by car_type")

fig.update_layout(
    xaxis_title="Engine power (ch)",
    yaxis_title="Car type",
    showlegend=False,
    height=600,
    margin=dict(l=120, r=40, t=60, b=40)
)

fig.show()

### Outliers removal

`engine_power` has outliers: values such as 0 ch or 25 ch are clearly improbable, and more generally values below 70ch seem suspect. Values above 300 ch are candidates for outliers.
We decide to remove values of engine_power below 1st percentile and above 99th percentile.


In [18]:
p1_engine_power = df['engine_power'].quantile(0.01)
p99_engine_power = df['engine_power'].quantile(0.99)
print(f"Engine power P1: {p1_engine_power}")
print(f"Engine power P99: {p99_engine_power}")

print(f"Engine power below {p1_engine_power} ch")
display(df.loc[df['engine_power']<p1_engine_power])

print(f"Engine power above {p99_engine_power} ch")
display(df.loc[df['engine_power']>p99_engine_power])

Engine power P1: 85.0
Engine power P99: 240.0
Engine power below 85.0 ch


,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
1136,BMW,22871,80,petrol,black,estate,True,False,False,False,False,False,True,117
1796,Porsche,152328,25,hybrid_petrol,black,hatchback,False,True,False,False,False,False,True,142
1804,Volkswagen,179307,70,diesel,blue,hatchback,False,True,False,False,False,False,True,91
1847,Volkswagen,100398,70,diesel,white,hatchback,False,True,False,False,False,True,True,103
1895,Porsche,26542,75,electro,grey,hatchback,False,True,False,False,False,False,True,145
1925,Porsche,152470,25,hybrid_petrol,black,hatchback,False,True,False,False,False,False,True,124
1983,Volkswagen,57344,70,diesel,grey,hatchback,False,True,False,False,False,False,True,109
1988,Volkswagen,150373,70,diesel,brown,hatchback,False,True,False,False,False,False,True,91
2001,Volkswagen,72527,70,diesel,silver,hatchback,False,False,False,False,False,False,True,96
2075,BMW,42273,80,petrol,black,hatchback,False,False,False,False,False,False,True,116


Engine power above 240.0 ch


,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
17,Peugeot,24521,270,petrol,grey,convertible,True,False,False,False,False,False,True,96
37,Peugeot,24452,270,petrol,grey,convertible,True,False,False,False,False,False,True,82
62,Renault,45536,250,petrol,white,coupe,True,True,True,False,True,True,True,197
67,Peugeot,29925,309,petrol,silver,coupe,True,True,False,False,True,True,True,217
72,Citroën,69410,317,petrol,white,coupe,True,True,False,False,False,True,True,232
73,Peugeot,170550,309,petrol,grey,coupe,True,True,False,False,True,False,True,167
93,Peugeot,99283,309,petrol,silver,coupe,False,False,False,False,True,False,True,169
139,Peugeot,169970,309,petrol,grey,coupe,True,True,False,False,True,False,True,189
954,BMW,184733,280,diesel,silver,estate,True,True,False,True,True,True,True,157


In [19]:
# Outliers removal: engine power
mask = (df['engine_power']<p1_engine_power) | (df['engine_power']>p99_engine_power)
print(f"Number of outliers to remove: {len(df.loc[mask])}")
df = df.loc[~mask]
print(f"Number of rows of the dataset after engine_power outliers' removal: {len(df)}")

# Outliers removal: mileage
# We keep it minimal and decide to drop negative values and the highest one (>1M km)
mask = (df['mileage']<0) | (df['mileage']>500000)
print(f"Number of outliers to remove: {len(df.loc[mask])}")
df = df.loc[~mask]
print(f"Number of rows of the dataset after mileage outliers' removal: {len(df)}")

Number of outliers to remove: 79
Number of rows of the dataset after engine_power outliers' removal: 4764
Number of outliers to remove: 2
Number of rows of the dataset after mileage outliers' removal: 4762


### Categorical variables

In [20]:
categorical_cols = ['model_key', 'fuel', 'paint_color',
       'car_type', 'private_parking_available', 'has_gps',
       'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
       'has_speed_regulator', 'winter_tires']


# Inspect values taken by model_key, fuel, paint_color, car_type
for var in categorical_cols:
       print(f"Variable: {var}\n")
       print(f"Nb of unique values: {df[var].nunique()}")
       print(f"Top 10 values: {df[var].value_counts(dropna=False).head(10)}\n")
       
       # Df with values, count and %
       value_counts = df[var].value_counts(dropna=False).reset_index()
       value_counts.columns = [var, 'count']
       value_counts['percentage'] = (value_counts['count'] / value_counts['count'].sum()) * 100
       value_counts['percentage'] = value_counts['percentage'].round(2)
       display(value_counts)
       print("\n" + "="*50 + "\n")

Variable: model_key

Nb of unique values: 25
Top 10 values: model_key
Citroën       966
Renault       909
BMW           811
Peugeot       633
Audi          526
Nissan        274
Mitsubishi    224
Mercedes       97
Volkswagen     50
Toyota         49
Name: count, dtype: int64



,model_key,count,percentage
0,Citroën,966,20.29
1,Renault,909,19.09
2,BMW,811,17.03
3,Peugeot,633,13.29
4,Audi,526,11.05
5,Nissan,274,5.75
6,Mitsubishi,224,4.70
7,Mercedes,97,2.04
8,Volkswagen,50,1.05
9,Toyota,49,1.03




Variable: fuel

Nb of unique values: 4
Top 10 values: fuel
diesel           4594
petrol            163
hybrid_petrol       4
electro             1
Name: count, dtype: int64



,fuel,count,percentage
0,diesel,4594,96.47
1,petrol,163,3.42
2,hybrid_petrol,4,0.08
3,electro,1,0.02




Variable: paint_color

Nb of unique values: 10
Top 10 values: paint_color
black     1605
grey      1159
blue       700
white      527
brown      339
silver     319
red         50
beige       41
green       16
orange       6
Name: count, dtype: int64



...

,has_getaround_connect,count,percentage
0,False,2562,53.8
1,True,2200,46.2




Variable: has_speed_regulator

Nb of unique values: 2
Top 10 values: has_speed_regulator
False    3627
True     1135
Name: count, dtype: int64



,has_speed_regulator,count,percentage
0,False,3627,76.17
1,True,1135,23.83




Variable: winter_tires

Nb of unique values: 2
Top 10 values: winter_tires
True     4437
False     325
Name: count, dtype: int64



,winter_tires,count,percentage
0,True,4437,93.18
1,False,325,6.82


Model key has 28 different modalities: 
7 brands have almost 100 observations or higher and account for 93% of the dataset. 
The top 3 brands are Citroen (20%), Renault (19%), BMV (17%) and account for more than half of the dataset.
21 brands are less represented in the dataset (under 50 observations, less than 7% of the dataset).
Among them we find rare brands for a grand public location, such as Lamborghini.
The flop 3 brands are Porsche, Honda and Mazda.

The fuel type is dominated by diesel (96%). Petrol cars account for 3.4% of the dataset. Hybrid petrol and electric are marginal (0.1% of the database).

The car color has 10 modalities.
The top 3 colors are black (34%), grey (24%) and blue (15%).
The flop 3 colors are beige (0.9%), green (0.3%) and orange (0.1%).
The top 5 colors account for 90% of the dataset (black, grey, blue, white, brown).

The car type has 8 modalites.
4 types account for 94% of the dataset: estate (34%), sedan (24%), suv (22%), hatchback (14%).
The 4 others are very specific and represent around 6% of the dataset: subcompact, coupe, convertible, van.

Private parking is available in 55% of the cases.
79% of car offers have GPS.
80% of them have air conditioning.
80% of them are automatic cars.
Less than a half has Get Around Connect check-in type.
76% of them has speed regulator.
93% of them has winter tires.

In [21]:
def plot_top_categories(df, col, top_n=5):
    
    # Counts
    counts = df[col].value_counts(dropna=False)
    
    # "Other" category which regroups rare modalities
    top_categories = counts.nlargest(top_n).copy()
    other_count = counts.sum() - top_categories.sum()
    if other_count > 0:
        top_categories['other'] = other_count
    
    # Percents
    total = top_categories.sum()
    percents = (top_categories / total * 100).round(0).values

    # Results dataframe
    df_plot = top_categories.reset_index()
    df_plot.columns = ['category', 'count']
    df_plot['percent'] = percents
    df_plot = df_plot.sort_values('count', ascending=False)
    
    fig = px.bar(
        df_plot,
        y='count',
        x='category',
        #orientation='h',
        title=f"Repartition of {col}",
        labels={'x': 'Nb cars', 'y': col},
        text_auto=True,
        width=700
    )
    fig.update_traces(
        texttemplate='%{y}<br> (%{customdata[0]}%)', 
        #textposition='outside', 
        customdata=df_plot[['percent']]
    )
    fig.update_layout(title={
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    })
    fig.show()

In [22]:
plot_top_categories(df, 'model_key', top_n=7)

In [23]:
plot_top_categories(df, 'fuel', top_n=2)

In [24]:
plot_top_categories(df, 'paint_color', top_n=5)

In [25]:
plot_top_categories(df, 'car_type', top_n=4)

In [26]:
binary_cols = ['private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
        'has_speed_regulator', 'winter_tires']

for var in binary_cols:
    fig = px.pie(
        df,
        names=var,
        width=400,
        height=400
    )

    fig.update_traces(
        texttemplate='%{percent:.1%}',
    )

    fig.update_layout(
        title_text=f'Repartition of {var}',
        title_x=0.48
    )

    fig.show()


### Bivariate analysis

We plot the distribution of the explanatory variables against the target variable, and the correlation matrix.

In [27]:
# Scatterplots

for var in numeric_cols:
    fig = px.scatter(
        df,
        x=var,
        y="rental_price_per_day",
        title=f"{var} vs rental price per day",
        labels={
            var: var,
            "rental_price_per_day": "Rental price per day"
        },
        width=800,
        height=500
    )
    fig.update_layout(
        title_x=0.5
    )
    fig.show()

In [28]:
# Correlation matrix
correlation_matrix = df[['mileage', 'engine_power', 'rental_price_per_day']].corr()

fig = px.imshow(correlation_matrix,
                text_auto='.2f',
                color_continuous_scale='Sunsetdark',
                width=600)

fig.update_layout(
    title='Correlation heatmap',
    title_x=0.5
)

fig.show()

Daily rental price is positively correlated with engine power (0.61) and is negatively correlated with mileage (-0.45).
On the scatterplots, for a given engine power (resp. mileage), the dispersion of daily rental seems high, suggesting that other factors influence the daily rental price.

In [29]:
for var in ['fuel', 'paint_color', 'car_type']:
    fig = px.box(
        df,
        x='rental_price_per_day',
        y=var,
        title=f"Distribution of daily rental price by {var}",
        height=450,
        width=900
    )
    fig.update_layout(
        yaxis_title=f"{var}",
        xaxis_title="Daily rental price",
        title_x=0.5,
        showlegend=False
    )
    fig.show()

In [30]:
# Box-plot for the top-10 model_key
top_model = df["model_key"].value_counts().nlargest(10).index
df_top_model = df.loc[df["model_key"].isin(top_model)]

fig = px.box(
    df_top_model,
    x="rental_price_per_day",
    y="model_key",
    color="model_key",
    height=700,
    labels={
        "model_key": "Brand",
        "rental_price_per_day": "Daily rental price"
    },
    title="Rental price distribution for the top-10 car brands"
)
fig.update_layout(
    title_x=0.5,
    showlegend=False
)
fig.show()

In [31]:
for var in binary_cols:
    fig = px.box(
        df,
        x='rental_price_per_day',
        y=var,
        title=f"Distribution of daily rental price by {var}",
        height=350,
        width=900
    )
    fig.update_layout(
        yaxis_title=f"{var}",
        xaxis_title="Daily rental price",
        title_x=0.5,
        showlegend=False
    )
    fig.show()

**Fuel type** seems to have a significant influence on rental price. 
Diesel and petrol cars tend to be priced lower than electric and hybrid petrol ones.
The price dispersion is higher for diesel, petrol and hybrid petrol cars, whereas it is almost null for electric cars.

**Paint color** has little impact on daily rental price; it can be considered as a secondary criteria.
The median daily price ranges around 110€ to 120€ for all colors, 
except for green (priced lower with a median price at 84€) and orange (priced higher with a median at 131€).

**Car type** seems to be an important factor of the daily rental price.
Coupes and SUV have the highest median price (149€ and 133€).
Subcompacts have the lowest median price (96€). 

**Car brand** can be considered as a driver of daily rental price. 
Volkswagen and Audi cars have higher median prices (130-145€) than such as Citroën and Peugeot (between 105-155€).
The price dispersion can be wide given the diversity of vehicles. 

**Automatic transmission** is a strong differentiating feature. Cars equipped with automatic transmission have a median daily price 28€ higher than those without.

**GPS**, **Get Around Connect**, **air conditioning**, **speed regulator** equipments and **private parking** also appear to be correlated with rental prices. Cars equipped with them have a median daily price between 13€ and 19€ higher than those without.

**Winter tires** do not seem to be correlated to rental prices: the median daily rental price is similar between cars equipped with and those which are not (around 120€). Indeed, this is more a safety equipment which can be mandatory according to road traffic legislation.

# 2. Preprocessings

In [33]:
# Binary variables: convert boolean values into numerical values
for var in binary_cols:
    df[var] = df[var].astype(bool).astype(int)

In [34]:
# Define features and target variable
features_list = ['model_key', 'mileage', 'engine_power', 'fuel', 'paint_color',
        'car_type', 'private_parking_available', 'has_gps',
        'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
        'has_speed_regulator', 'winter_tires']
target_variable = 'rental_price_per_day'

X = df.loc[:, features_list]
Y = df.loc[:, target_variable]

# Define numeric and categorical features
numeric_features = ['mileage', 'engine_power']
categorical_features = ['model_key', 'fuel', 'paint_color',
        'car_type', 'private_parking_available', 'has_gps',
        'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
        'has_speed_regulator', 'winter_tires']

# Divide dataset between train set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, shuffle=True)

In [35]:
def group_rare_categories(df, cols, min_freq=0.01):
    """
    Remplace les catégories rares (< min_freq) par 'other' dans les colonnes spécifiées.
    df : DataFrame
    cols : liste des colonnes à traiter
    min_freq : fréquence minimale pour qu'une catégorie soit conservée
    """
    df = df.copy()
    for col in cols:
        freq = df[col].value_counts(normalize=True)
        rares = freq[freq < min_freq].index
        df[col] = df[col].replace(rares, "other")
    return df

cols_with_rare_categories = ["paint_color", "fuel", "model_key"]

rare_transformer = FunctionTransformer(
    func=lambda X: group_rare_categories(X, cols=cols_with_rare_categories, min_freq=0.01),
    validate=False
)

In [36]:
# Create pipeline for numeric features
numeric_transformer = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
    ]
)

# Create pipeline for categorical features
categorical_transformer = Pipeline([
    ("rare", rare_transformer),
    ("ohe", OneHotEncoder(drop="first", handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


# 3. Global performance overview

We first train several models using their default hyperparameters: simple linear regression, regularized linear regression (Ridge and Lasso), random forest and XGBoost.
We use cross-validation on the default models in order to have a first view of their performance.  

Random forests and XG Boost seem to give the best results compared to linear regression (simple / regularized).

In [37]:
# Pipeline for multivariate linear regression
pipeline_linreg = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", LinearRegression())
])

pipeline_ridge = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", Ridge(alpha=1.0))
])

pipeline_lasso = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", Lasso(alpha=0.1))
])

# Pipeline for random forest
pipeline_rf = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", RandomForestRegressor(random_state=0))
])

# Pipeline for XG Boost
pipeline_xgb = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", XGBRegressor(objective='reg:squarederror', random_state=0))
])

In [38]:
models = {
    "Simple linear regression": pipeline_linreg,
    "Ridge regression": pipeline_ridge,
    "Lasso regression": pipeline_lasso,
    "Random forest": pipeline_rf,
    "XG Boost": pipeline_xgb
}

results_cv = []

cv = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = ['r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error']

results_cv = []

for name, model in models.items():
    scores = cross_validate(model, X, Y, cv=cv, scoring=scoring, return_train_score=False)
    
    results_cv.append({
        "Model": name,
        "Mean R²": scores['test_r2'].mean(),
        "Std R²": scores['test_r2'].std(),
        "Mean RMSE": -scores['test_neg_root_mean_squared_error'].mean(),
        "Std RMSE": scores['test_neg_root_mean_squared_error'].std(),
        "Mean MAE": -scores['test_neg_mean_absolute_error'].mean(),
        "Std MAE": scores['test_neg_mean_absolute_error'].std()
    })    

In [39]:
df_results_cv = pd.DataFrame(results_cv)
df_results_cv.sort_values("Mean RMSE", ascending=True).round(3)

,Model,Mean R²,Std R²,Mean RMSE,Std RMSE,Mean MAE,Std MAE
3,Random forest,0.744,0.032,16.388,1.341,10.514,0.299
4,XG Boost,0.740,0.025,16.516,1.092,10.642,0.218
1,Ridge regression,0.683,0.019,18.278,1.022,12.392,0.336
0,Simple linear regression,0.681,0.018,18.320,0.992,12.418,0.339
2,Lasso regression,0.678,0.022,18.392,1.092,12.400,0.341


# 4. Model training

We choose to test the following models:
- regularized linear regression: Ridge and Lasso;
- random forest;
- XG Boost model.

For each model:
- preprocessing and model fitting are combined in a pipeline;
- potential hyperparameters are tuned using GridSearch;
- assessment metrics are computed: R², RMSE, MAE.

For GridSearch, the metric we choose to minimize is **RMSE** (root mean squared error) because we want to avoid large prediction mistakes. RMSE gives the average prediction error, by penalizing larger errors more heavily than smaller ones.

To ensure results reproducibility, we set a random seed equal to 0 (`random_state=0`). Also, data is shuffled (meaning that the order of the observations is randomly rearranged) in order to avoid bias and ensure independent samples.

In [44]:
# Initialize dicts containing GridSearchCV results and best models
grid_results = {}
best_models = {}

## 4.1. Ridge regression

In [40]:
'''
param_grid_ridge = {
    "model__alpha": [0.01, 0.1, 1, 10, 100]
}
'''
param_grid_ridge = {
    "model__alpha": [5, 7.5, 10, 12.5, 15]
}

In [41]:
grid_ridge = GridSearchCV(
    pipeline_ridge,
    param_grid=param_grid_ridge,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
grid_ridge.fit(X_train, Y_train)

,estimator,"Pipeline(step...l', Ridge())])"
,param_grid,"{'model__alpha': [5, 7.5, ...]}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [ ]:
# best_score_ is the mean cross-validation score for the best combination of hyperparameters tested during the grid search
# best_params_ is a dictionary containing the combination of hyperparameters that gave the best cross-validation score during the grid search

# best_estimator_ is the model trained on all training data using best_params_, the best combination of hyperparameters found during the grid search. 
# It's the version of the model to use for predictions, if this model is selected.

print("Best score CV (RMSE)", -round(grid_ridge.best_score_, 3))
print("Best params", grid_ridge.best_params_)
print("Best estimator", grid_ridge.best_estimator_)

Best score CV (RMSE) 18.222
Best params {'model__alpha': 10}
Best estimator Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('rare',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000002A04AAC8E00>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                        

In [45]:
# Append results

best_models["Ridge"] = grid_ridge.best_estimator_

Y_pred = best_models["Ridge"].predict(X_test)

rmse_test = root_mean_squared_error(Y_test, Y_pred)
mae_test = mean_absolute_error(Y_test, Y_pred)
r2_test = r2_score(Y_test, Y_pred)

# index of the best combination of hyperparameters in cv_results_
best_idx = grid_ridge.best_index_

# standard deviation of CV score associated with the best combination of hyperparameters
rmse_cv_std = grid_ridge.cv_results_["std_test_score"][best_idx]

grid_results["Ridge"] = {
    "Best params": grid_ridge.best_params_,
    "CV RMSE mean": -grid_ridge.best_score_,
    "CV RMSE std": rmse_cv_std,
    "Test RMSE": rmse_test,
    "Test MAE": mae_test,
    "Test R2": r2_test
}

In [ ]:
# Print results
results_df = pd.DataFrame.from_dict(grid_results, orient="index")
print("Model comparison (CV scores):")
display(results_df.round(3))

Model comparison (CV scores):


,Best params,CV RMSE mean,CV RMSE std,Test RMSE,Test MAE,Test R2
Ridge,{'model__alpha': 10},18.222,0.707,18.97,12.719,0.676


## 4.2. Lasso

In [47]:
param_grid_lasso = {
    "model__alpha": [0.01, 0.1, 1, 5, 10, 20]
}

grid_lasso = GridSearchCV(
    pipeline_lasso,
    param_grid=param_grid_lasso,
    cv=cv,
    scoring="neg_root_mean_squared_error", 
    n_jobs=-1
)
grid_lasso.fit(X_train, Y_train)

,estimator,Pipeline(step...(alpha=0.1))])
,param_grid,"{'model__alpha': [0.01, 0.1, ...]}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [48]:
print("Best score CV (RMSE)", -round(grid_lasso.best_score_, 3))
print("Best params", grid_lasso.best_params_)
print("Best estimator", grid_lasso.best_estimator_)

Best score CV (RMSE) 18.326
Best params {'model__alpha': 0.01}
Best estimator Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('rare',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000002A04AAC8E00>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                      

In [49]:
# Append results

best_models["Lasso"] = grid_lasso.best_estimator_

Y_pred = best_models["Lasso"].predict(X_test)

# index of the best combination of hyperparameters in cv_results_
best_idx = grid_lasso.best_index_

# standard deviation of CV score associated with the best combination of hyperparameters
rmse_cv_std = grid_lasso.cv_results_["std_test_score"][best_idx]

rmse_test = root_mean_squared_error(Y_test, Y_pred)
mae_test = mean_absolute_error(Y_test, Y_pred)
r2_test = r2_score(Y_test, Y_pred)

grid_results["Lasso"] = {
    "Best params": grid_lasso.best_params_,
    "CV RMSE mean": -grid_lasso.best_score_,
    "CV RMSE std": rmse_cv_std,
    "Test RMSE": rmse_test,
    "Test MAE": mae_test,
    "Test R2": r2_test
}

In [50]:
# Display results
results_df = pd.DataFrame.from_dict(grid_results, orient="index")
print("Model comparison (CV scores):")
display(results_df.round(3))

Model comparison (CV scores):


,Best params,CV RMSE mean,CV RMSE std,Test RMSE,Test MAE,Test R2
Ridge,{'model__alpha': 10},18.222,0.707,18.970,12.719,0.676
Lasso,{'model__alpha': 0.01},18.326,0.627,18.965,12.741,0.676


## 4.3. Random forest

In [51]:
param_grid_rf = {
    "model__n_estimators": [100, 200],
    "model__max_depth": [5, 10, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
    "model__max_features": ['sqrt', 'log2']
}

grid_rf = GridSearchCV(
    pipeline_rf,
    param_grid=param_grid_rf,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

grid_rf.fit(X_train, Y_train)

,estimator,Pipeline(step...om_state=0))])
,param_grid,"{'model__max_depth': [5, 10, ...], 'model__max_features': ['sqrt', 'log2'], 'model__min_samples_leaf': [1, 2, ...], 'model__min_samples_split': [2, 5, ...], ...}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [52]:
print("Best score", -round(grid_rf.best_score_, 3))
print("Best params", grid_rf.best_params_)
print("Best estimator", grid_rf.best_estimator_)

Best score 16.674
Best params {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Best estimator Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('rare',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000002A04AAC8E00>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                  

In [53]:
best_models["Random forest"] = grid_rf.best_estimator_

Y_pred = best_models["Random forest"].predict(X_test)

rmse_test = root_mean_squared_error(Y_test, Y_pred)
mae_test = mean_absolute_error(Y_test, Y_pred)
r2_test = r2_score(Y_test, Y_pred)

# Index of the best combination of hyperparameters in cv_results_
best_idx = grid_rf.best_index_

# Standard deviation of CV score associated with the best combination of hyperparameters
rmse_cv_std = grid_rf.cv_results_["std_test_score"][best_idx]

grid_results["Random forest"] = {
    "Best params": grid_rf.best_params_,
    "CV RMSE mean": -grid_rf.best_score_,
    "CV RMSE std": rmse_cv_std,
    "Test RMSE": rmse_test,
    "Test MAE": mae_test,
    "Test R2": r2_test
}

In [54]:
# Print results
results_df = pd.DataFrame.from_dict(grid_results, orient="index")
print("Model comparison (CV scores):")
display(results_df.round(3))

Model comparison (CV scores):


,Best params,CV RMSE mean,CV RMSE std,Test RMSE,Test MAE,Test R2
Ridge,{'model__alpha': 10},18.222,0.707,18.970,12.719,0.676
Lasso,{'model__alpha': 0.01},18.326,0.627,18.965,12.741,0.676
Random forest,"{'model__max_depth': None, 'model__max_feature...",16.674,1.005,17.372,10.794,0.728


## 4.4. XG Boost

In [ ]:
param_grids = {
    "model__n_estimators": [100, 200, 300],     # Number of trees. Default:100
    "model__learning_rate": [0.05, 0.1, 0.3],   # Default: 0.3. Controls how much each new tree contributes to correcting errors from previous trees.
    "model__max_depth": [4, 5, 6],              # Maximum depth of each tree. Default:6
    "model__min_child_weight": [1, 3, 5],       # Minimum sum of instance weights needed in a child. Default=1
    "model__subsample": [0.8, 1.0],             # Fraction of training samples used per tree. Default= 1
    "model__colsample_bytree": [0.8, 1.0],      # Fraction of features used per tree. Default= 1
    "model__reg_alpha": [0, 1, 3],              # L1 regularization term. Default= 0
    "model__reg_lambda": [1, 3]                 # L2 regularization term. Default= 1
}

In [62]:
# GridSearchCV
grid_xgb = GridSearchCV(
    pipeline_xgb,
    param_grid=param_grids,
    cv=cv,
    scoring='neg_root_mean_squared_error',  
    verbose=2,
    n_jobs=-1
)

# Train model
grid_xgb.fit(X_train, Y_train)

Fitting 5 folds for each of 1944 candidates, totalling 9720 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_grid,"{'model__colsample_bytree': [0.8, 1.0], 'model__learning_rate': [0.05, 0.1, ...], 'model__max_depth': [4, 5, ...], 'model__min_child_weight': [1, 3, ...], ...}"
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...)]"


In [64]:
print("Best score", -round(grid_xgb.best_score_, 3))
print("Best params", grid_xgb.best_params_)
print("Best estimator", grid_xgb.best_estimator_)

Best score 15.888
Best params {'model__colsample_bytree': 0.8, 'model__learning_rate': 0.05, 'model__max_depth': 6, 'model__min_child_weight': 3, 'model__n_estimators': 300, 'model__reg_alpha': 3, 'model__reg_lambda': 3, 'model__subsample': 0.8}
Best estimator Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['mileage', 'engine_power']),
                                                 ('cat',
                                                  Pipeline(steps=[('rare',
                                                                   FunctionTransformer(func=<function <lambda> at 0x000002A04AAC8E00>)),
                                                                  ('ohe',
                                                            

In [65]:
best_models["XG Boost"] = grid_xgb.best_estimator_

Y_pred = best_models["XG Boost"].predict(X_test)

rmse_test = root_mean_squared_error(Y_test, Y_pred)
mae_test = mean_absolute_error(Y_test, Y_pred)
r2_test = r2_score(Y_test, Y_pred)

# Index of the best combination of hyperparameters in cv_results_
best_idx = grid_xgb.best_index_

# Standard deviation of CV score associated with the best combination of hyperparameters
rmse_cv_std = grid_xgb.cv_results_["std_test_score"][best_idx]

grid_results["XG Boost"] = {
    "Best params": grid_xgb.best_params_,
    "CV RMSE mean": -grid_xgb.best_score_,
    "CV RMSE std": rmse_cv_std,
    "Test RMSE": rmse_test,
    "Test MAE": mae_test,
    "Test R2": r2_test
}

# 5. Selection of the best model

In [66]:
# Comparison of the models
df_results = pd.DataFrame.from_dict(grid_results, orient="index").reset_index().rename(columns={"index":"Model"})
display(df_results.round(3))

,Model,Best params,CV RMSE mean,CV RMSE std,Test RMSE,Test MAE,Test R2
0,Ridge,{'model__alpha': 10},18.222,0.707,18.970,12.719,0.676
1,Lasso,{'model__alpha': 0.01},18.326,0.627,18.965,12.741,0.676
2,Random forest,"{'model__max_depth': None, 'model__max_feature...",16.674,1.005,17.372,10.794,0.728
3,XG Boost,"{'model__colsample_bytree': 0.8, 'model__learn...",15.888,0.599,16.339,10.334,0.760


In [67]:
fig = px.scatter(
    df_results,
    x="Model",
    y="CV RMSE mean",
    error_y="CV RMSE std",
    title="Comparison of models performance (CV RMSE mean ± std)",
    labels={
        "CV RMSE mean": "CV RMSE",
        "CV RMSE std": "Std Dev"
    },
    width=700
)

fig.update_traces(marker=dict(size=10))
fig.update_layout(    
        title=dict(
        text="Comparison of models performance (CV RMSE mean ± std)",
        x=0.5,
        xanchor='center'
    ),)
fig.show()


Models based on decision trees outperform linear models on this dataset.
Based on CV mean RMSE, XGBoost is the best, followed by random forest, then Ridge and Lasso.

Looking at the intervals of mean +/- standard deviation across cross-validation folds:
- Ridge and Lasso overlap a lot: their performances are close
- random forest and XGBoost have less overlap.

Based on cross-validation results, XGBoost achieves the lowest mean RMSE (15.888) with a reasonably low standard deviation (0.6). It indicates that the model is likely to make more reliable predictions than the other models (a good predictive performance with more stable behavior between folds).
On the test set, XGBoost achieves the best performance across all metrics (MAE, R², and RMSE). XGBoost not only performs well on cross-validation but also generalizes best to unseen data compared to random forest and linear models (Ridge and Lasso).

Random forest is the second best model in terms of CV mean RMSE (between linear models and XGBoost); however it has higher variability (the highest std among all the models).

Linear models such as Ridge and Lasso - more simple to interpret than XGBoost or random forest - give the most stable predictions (lower std) but are less accurate (higher CV mean RMSE).

So we decide to select XGBoost as the best model among the four.

In [68]:
# Final model
final_model = best_models["XG Boost"]

# 6. Best model results and evaluation

### Evaluate performances on training and test set

In [69]:
Y_train_pred = final_model.predict(X_train)
Y_test_pred = final_model.predict(X_test)

In [72]:
def evaluate(y_true, y_pred):
    rmse = round(np.sqrt(mean_squared_error(y_true, y_pred)), 3)
    mae = round(mean_absolute_error(y_true, y_pred), 3)
    r2 = round(r2_score(y_true, y_pred), 3)
    return rmse, mae, r2

rmse_train, mae_train, r2_train = evaluate(Y_train, Y_train_pred)
rmse_test, mae_test, r2_test = evaluate(Y_test, Y_test_pred)

print("Train RMSE:", rmse_train, 
    "MAE:", mae_train, 
    "R2:", r2_train)
print("Test RMSE:", rmse_test, 
    "MAE:", mae_test,
    "R2:", r2_test)

Train RMSE: 10.697 MAE: 7.294 R2: 0.89
Test RMSE: 16.339 MAE: 10.334 R2: 0.76


### Residuals

In [77]:
residuals = Y_test - Y_test_pred

df_resid = pd.DataFrame({
    "Observed": Y_test,
    "Predicted": Y_test_pred,
    "Residual": residuals
})

# Scatter plot : predicted vs observed
fig = px.scatter(
    df_resid,
    x="Observed",
    y="Predicted",
    hover_data=["Residual"],
    title="Predicted vs observed (test set)",
    labels={"Observed": "Observed values", "Predicted": "Predicted values"}
)

fig.add_shape(
    type="line",
    x0=df_resid["Observed"].min(),
    x1=df_resid["Observed"].max(),
    y0=df_resid["Observed"].min(),
    y1=df_resid["Observed"].max(),
    line=dict(color="red", dash="dash")
)

fig.update_layout(title_x=0.5)
fig.show()

The points are generally close to the red diagonal line (which represents perfect prediction line y = x), meaning the model makes overall good predictions.

Some extreme values/outliers are poorly predicted: some cases with a very low observed price (10€/day) and a case with a very high value (>370€/day). These cases need to be investigated further.

In [80]:
# Residuals scatter plot 
fig = px.scatter(
    df_resid,
    x="Predicted",
    y="Residual",
    title="Residuals vs predicted values (test set)",
    labels={"Predicted": "Predicted values", "Residual": "Residuals"}
)

fig.add_shape(
    type="line",
    x0=df_resid["Predicted"].min(),
    x1=df_resid["Predicted"].max(),
    y0=0,
    y1=0,
    line=dict(color="red", dash="dash")
)

fig.update_layout(title_x=0.5)
fig.show()


The dispersion of residuals is globally stable across the predicted values, and centered around zero.
There is one extreme case with a residual above 200 which needs to be investigated.

### Feature importance

In [81]:
# Retrieve feature names after preprocessing
preprocessor = final_model.named_steps['preprocessing']
cat_pipeline = preprocessor.named_transformers_['cat']
ohe = cat_pipeline.named_steps['ohe']
cat_feature_names = ohe.get_feature_names_out(categorical_features)
num_feature_names = numeric_features
feature_names = list(num_feature_names) + list(cat_feature_names)

# Retrieve the XGBoost model
regressor = final_model.named_steps['model']

# Features importances
importances = regressor.feature_importances_

feat_imp_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

display(feat_imp_df.round(3))

,Feature,Importance
30,automatic_car_1,0.167
1,engine_power,0.098
5,model_key_Mitsubishi,0.066
31,has_getaround_connect_1,0.058
28,has_gps_1,0.053
0,mileage,0.043
6,model_key_Nissan,0.043
13,fuel_petrol,0.041
3,model_key_Citroën,0.038
25,car_type_suv,0.032


In [95]:
fig = px.bar(
    feat_imp_df,
    x="Importance",
    y="Feature",
    orientation="h",
    title=f"Features importances",
    width=900,
    height=700
)

fig.update_layout(
    title_x=0.5,
    yaxis=dict(autorange="reversed"))

fig.show()
